### **Libraries**

Some of the needed libraries for this project.

In [1]:
import os
import openai
from openai import OpenAI

In [2]:
import pathlib
import textwrap

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") # OpenAI API key

#### **Define a client**

Created an instance of the OpenAI model

In [4]:
# define an instance of OpenAI model
client = OpenAI(api_key=OPENAI_API_KEY)

#### **Audio transcription**

Transcribed the audio file.

In [5]:
audio_file= open("multiple.mp3", "rb")

In [6]:
speech_to_text = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file
)

In [7]:
to_markdown(speech_to_text.text)

> On the 21st of July 2024, I'll be having a meeting with Alex, which will start by 7 p.m. and end by 8 p.m. The event will take place on Google Meet. Set the reminder for 15 minutes. And on the 28th of July 2024, I'll have another meeting with Adam by 3 p.m. and it will end by 4 p.m. The event is going to take place on Google Meet. Set the reminder for 10 minutes.

### **Conversational chat**

Using conversational chat to extract event related data.

In [8]:
system_prompt = f"""
You are an assistant that creates a well crafted schedule from text. 
Extract all events related data from the provided text and put it in JSON format.


{speech_to_text.text}
"""

In [9]:
from langchain_openai import ChatOpenAI

In [10]:
chatmodel = ChatOpenAI(
    model='gpt-3.5-turbo',
    api_key=OPENAI_API_KEY,
    temperature=0.3,
)

In [11]:
format_chat  = chatmodel.invoke(system_prompt)


In [12]:
to_markdown(format_chat.content)

> {
>   "events": [
>     {
>       "date": "21st of July 2024",
>       "event": "Meeting with Alex",
>       "start_time": "7:00 p.m.",
>       "end_time": "8:00 p.m.",
>       "location": "Google Meet",
>       "reminder": "15 minutes"
>     },
>     {
>       "date": "28th of July 2024",
>       "event": "Meeting with Adam",
>       "start_time": "3:00 p.m.",
>       "end_time": "4:00 p.m.",
>       "location": "Google Meet",
>       "reminder": "10 minutes"
>     }
>   ]
> }

#### **Calendar file creation**

Extraction of event data.

In [76]:
# user prompt
user_prompt = f"From the text {speech_to_text.text} create a calendar-like structure."

calendar_file = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {'role': 'user', 'content': user_prompt}
    ],
    tools=schedule_tool,
    tool_choice='required',
    temperature=0.3
)

In [77]:
print(calendar_file)

ChatCompletion(id='chatcmpl-9jrIiL7pqKaH0J8JLk9y8lEp4ZlYJ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_xwkf0K8CekVTlc92riUaIa4K', function=Function(arguments='{"date":["2024-07-14"],"start_time":["19:00"],"end_time":["21:00"],"event":["Synchronized learning with Google"],"location":["Zoom"],"event_details":["Synchronized learning with Google on Zoom"],"reminder":["18:45"]}', name='schedule_events'), type='function')]))], created=1720716676, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_d33f7b429e', usage=CompletionUsage(completion_tokens=68, prompt_tokens=244, total_tokens=312))


In [78]:
if(calendar_file.choices[0].message.tool_calls):
  #If tool call was invoked extract the name of the specific function used
  function_name = calendar_file.choices[0].message.tool_calls[0].function.name
  #Print the name of the function called
  #print("called the function: " + function_name)
  # Extract response to user query
  result = calendar_file.choices[0].message.tool_calls[0].function.arguments
else:
  print("didn't call a function")
  result = calendar_file.choices[0].message.content
  
to_markdown(result)

> {"date":["2024-07-14"],"start_time":["19:00"],"end_time":["21:00"],"event":["Synchronized learning with Google"],"location":["Zoom"],"event_details":["Synchronized learning with Google on Zoom"],"reminder":["18:45"]}

#### **String output to JSON file**

In [79]:
# Used to create a JSON file
import json

In [80]:
# converts string to dictionary
result_dict = json.loads(result)

In [81]:
# 
events_dict = {}
# added the extracted events to the dictionary
events_dict['events'] = [result_dict]
print(events_dict)


{'events': [{'date': ['2024-07-14'], 'start_time': ['19:00'], 'end_time': ['21:00'], 'event': ['Synchronized learning with Google'], 'location': ['Zoom'], 'event_details': ['Synchronized learning with Google on Zoom'], 'reminder': ['18:45']}]}


In [82]:

# Define the file path
json_file_path = 'events.json'

# Write the result to the JSON file
with open(json_file_path, 'w') as file:
    json.dump(events_dict, file, indent=4)

#### **JSON file to .ics file**

In [83]:
from icalendar import Calendar, Event, Alarm
from datetime import datetime
from pytz import UTC  # This is required for the 'dt' fields in icalendar

# Path to your JSON file
json_file_path = 'events.json'

# Read and load the JSON data from file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Create an iCalendar file
cal = Calendar()

for event in data['events']:
    # Create an event
    ical_event = Event()
    
    # Assuming the first item in each list is the relevant one
    start_date_str = event['date'][0] + ' ' + event['start_time'][0]
    end_date_str = event['date'][0] + ' ' + event['end_time'][0]
    reminder_str = event['date'][0] + ' ' + event['reminder'][0]
    
    # Convert to datetime objects
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d %H:%M').replace(tzinfo=UTC)
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d %H:%M').replace(tzinfo=UTC)
    reminder = datetime.strptime(reminder_str, '%Y-%m-%d %H:%M').replace(tzinfo=UTC)
    
    # Add details to the event
    ical_event.add('summary', event['event'][0])
    ical_event.add('dtstart', start_date)
    ical_event.add('dtend', end_date)
    
    # Optional: Add location and description if available
    if event['location']:
        ical_event.add('location', event['location'][0])
    if event['event_details']:
        ical_event.add('description', event['event_details'][0])

    # Create a reminder (alarm)
    alarm = Alarm()
    alarm.add('action', 'DISPLAY')
    alarm.add('description', 'Reminder')
    alarm.add('trigger', start_date - reminder)  
    
    # Add the alarm to the event
    ical_event.add_component(alarm)
    
    # Add the event to the calendar
    cal.add_component(ical_event)

# Save the iCalendar file
file_name = 'events.ics'
with open(file_name, 'wb') as f:
    f.write(cal.to_ical())

print(f"iCalendar file '{file_name}' created successfully.")

iCalendar file 'events.ics' created successfully.


#### **Writing the .ics file to calendar**

In [84]:
import os
from abc import ABC, abstractmethod
from icalendar import Calendar

# Step 1: Read and parse the ICS file
def read_ics(file_name):
    with open(file_name, 'r') as ics_file:
        return Calendar.from_ical(ics_file.read())

# Step 2: Define an abstract base class for calendar services
class CalendarService(ABC):
    def __init__(self, credentials):
        self.credentials = credentials

    @abstractmethod
    def authenticate(self):
        pass

    @abstractmethod
    def create_event(self, event):
        pass

# Step 3: Implement Google Calendar handler
class GoogleCalendarService(CalendarService):
    def authenticate(self):
        from google.oauth2.credentials import Credentials
        from google_auth_oauthlib.flow import InstalledAppFlow
        from google.auth.transport.requests import Request
        from googleapiclient.discovery import build

        SCOPES = ['https://www.googleapis.com/auth/calendar']
        
        creds = None
        if os.path.exists('token.json'):
            creds = Credentials.from_authorized_user_file('token.json', SCOPES)
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.credentials, SCOPES)
                creds = flow.run_local_server(port=0)
            with open('token.json', 'w') as token:
                token.write(creds.to_json())
        
        self.service = build('calendar', 'v3', credentials=creds)

    def create_event(self, event):
        google_event = {
            'summary': str(event.get('summary')),
            'start': {'dateTime': event.get('dtstart').dt.isoformat(), 'timeZone': 'UTC'},
            'end': {'dateTime': event.get('dtend').dt.isoformat(), 'timeZone': 'UTC'}
        }
        self.service.events().insert(calendarId='primary', body=google_event).execute()

# Step 4: Implement Outlook Calendar handler
class OutlookCalendarService(CalendarService):
    def authenticate(self):
        from O365 import Account, FileSystemTokenBackend
        
        token_backend = FileSystemTokenBackend(token_path='.')
        self.account = Account(self.credentials, token_backend=token_backend)

        if not self.account.is_authenticated:
            self.account.authenticate(scopes=['https://graph.microsoft.com/Calendars.ReadWrite'])

    def create_event(self, event):
        schedule = self.account.schedule()
        calendar = schedule.get_default_calendar()
        
        new_event = calendar.new_event()
        new_event.subject = str(event.get('summary'))
        new_event.start = event.get('dtstart').dt
        new_event.end = event.get('dtend').dt
        new_event.save()

# Step 5: General function to add events to calendar
def add_events_to_calendar(service, ics_file_path):
    calendar = read_ics(ics_file_path)
    for component in calendar.walk():
        if component.name == "VEVENT":
            service.create_event(component)

# Example Usage:
# Initialize the Google Calendar service
google_service = GoogleCalendarService('credential.json')
google_service.authenticate()
add_events_to_calendar(google_service, file_name)

# Initialize the Outlook Calendar service
#outlook_service = OutlookCalendarService(('your_client_id', 'your_client_secret'))
#outlook_service.authenticate()
#add_events_to_calendar(outlook_service, file_name)


simple sequential chain to get and parse the 